# Analysis Flow

This notebook demonstrates how an Automate Flow can be used to perform analysis. 
We use a flow that combines Globus Transfer and funcX to move data and then execute an analysis code on the data.

In [1]:
from globus_automate_client import (create_flows_client, graphviz_format, state_colors_for_log,
                                    create_action_client, 
                                    create_flows_client)

import time
import json
import sys
import os

from funcx.sdk.client import FuncXClient

CLIENT_ID = "e6c75d97-532a-4c88-b031-8584a319fa3e"

## Create an analysis function

Use funcX to register a simple analysis function.

In [2]:
fxc = FuncXClient()

In [3]:
def file_size(data):
    """Return the size of a file"""
    import os
    return os.path.getsize(data['pathname'])

func_uuid = fxc.register_function(file_size)
print(func_uuid)

df1b5032-eaee-4f02-a398-12456918b22e


In [4]:
payload = {'pathname': '/etc/hostname'}
tutorial_endpoint = '4b116d3c-1703-4f8f-9f6f-39921e5864df' # Public tutorial endpoint
res = fxc.run(payload, endpoint_id=tutorial_endpoint, function_id=func_uuid)
print(res)

7b3f28df-22a5-4c3c-9ed2-78c35cf723e8


In [5]:
fxc.get_result(res)

26

## Create the flow

In [6]:
flow_definition = {
  "Comment": "An analysis flow",
  "StartAt": "Transfer",
  "States": {
    "Transfer": {
      "Comment": "Initial transfer",
      "Type": "Action",
      "ActionUrl": "https://actions.automate.globus.org/transfer/transfer",
      "ActionScope": "https://auth.globus.org/scopes/actions.globus.org/transfer/transfer",
      "Parameters": {
        "source_endpoint_id.$": "$.input.source_endpoint", 
        "destination_endpoint_id.$": "$.input.dest_endpoint",
        "transfer_items": [
          {
            "source_path.$": "$.input.source_path",
            "destination_path.$": "$.input.dest_path",
            "recursive": False
          }
        ]
      },
      "ResultPath": "$.Transfer1Result",
      "WaitTime": 600,
      "Next": "Analyze"
    },
    "Analyze": {
      "Comment": "Run a funcX function",
      "Type": "Action",
      "ActionUrl": "https://api.funcx.org/automate",
      "ActionScope": "https://auth.globus.org/scopes/facd7ccc-c5f4-42aa-916b-a0e270e2c2a9/automate2",
      "Parameters": {
          "tasks": [{
            "endpoint.$": "$.input.fx_ep",
            "func.$": "$.input.fx_id",
            "payload": {
                "pathname.$": "$.input.pathname"
            }
        }]
      },
      "ResultPath": "$.AnalyzeResult",
      "WaitTime": 600,
      "Next": "Transfer2"
    },
    "Transfer2": {
      "Comment": "Return transfer",
      "Type": "Action",
      "ActionUrl": "https://actions.automate.globus.org/transfer/transfer",
      "ActionScope": "https://auth.globus.org/scopes/actions.globus.org/transfer/transfer",
      "Parameters": {
        "source_endpoint_id.$": "$.input.dest_endpoint", 
        "destination_endpoint_id.$": "$.input.source_endpoint",
        "transfer_items": [
          {
            "source_path.$": "$.input.dest_path",
            "destination_path.$": "$.input.result_path",
            "recursive": False
          }
        ]
      },
      "ResultPath": "$.Transfer2Result",
      "WaitTime": 600,
      "End": True
    },
  }
}


Register the flow

In [7]:
flows_client = create_flows_client(CLIENT_ID)
flow = flows_client.deploy_flow(flow_definition, title="Stills process workflow")
flow_id = flow['id']
print(flow)
flow_scope = flow['globus_auth_scope']
print(f'Newly created flow with id:\n{flow_id}\nand scope:\n{flow_scope}')

GlobusHTTPResponse({'action_url': 'https://flows.automate.globus.org/flows/4b42bc90-951a-44ea-b761-93c3bf4f7b60', 'administered_by': [], 'api_version': '1.0', 'definition': {'Comment': 'An analysis flow', 'StartAt': 'Transfer', 'States': {'Analyze': {'ActionScope': 'https://auth.globus.org/scopes/facd7ccc-c5f4-42aa-916b-a0e270e2c2a9/automate2', 'ActionUrl': 'https://api.funcx.org/automate', 'Comment': 'Run a funcX function', 'Next': 'Transfer2', 'Parameters': {'tasks': [{'endpoint.$': '$.input.fx_ep', 'func.$': '$.input.fx_id', 'payload': {'pathname.$': '$.input.pathname'}}]}, 'ResultPath': '$.AnalyzeResult', 'Type': 'Action', 'WaitTime': 600}, 'Transfer': {'ActionScope': 'https://auth.globus.org/scopes/actions.globus.org/transfer/transfer', 'ActionUrl': 'https://actions.automate.globus.org/transfer/transfer', 'Comment': 'Initial transfer', 'Next': 'Analyze', 'Parameters': {'destination_endpoint_id.$': '$.input.dest_endpoint', 'source_endpoint_id.$': '$.input.source_endpoint', 'transfe

In [8]:
src_ep = 'ddb59aef-6d04-11e5-ba46-22000b92c6ec' # EP1
dest_ep = 'ddb59af0-6d04-11e5-ba46-22000b92c6ec' # EP2
filename = 'test.txt'

flow_input = {
    "input": {
        "source_endpoint": src_ep,
        "source_path": f"/~/{filename}",
        "dest_endpoint": dest_ep,
        "dest_path": f"/~/{filename}",
        "result_path": f"/~/out_{filename}",
        "fx_id": func_uuid,
        "fx_ep": tutorial_endpoint,
        "pathname": '/etc/hostname'
    }
}

Run the flow and wait for it to complete

Note: This will try to transfer a file called "test.txt" from your Globus Tutorial Endpoint 1.

In [9]:
flow_action = flows_client.run_flow(flow_id, flow_scope, flow_input)
print(flow_action)
flow_action_id = flow_action['action_id']
flow_status = flow_action['status']
print(f'Flow action started with id: {flow_action_id}')
while flow_status == 'ACTIVE':
    time.sleep(10)
    flow_action = flows_client.flow_action_status(flow_id, flow_scope, flow_action_id)
    flow_status = flow_action['status']
    print(f'Flow status: {flow_status}')

Login Here:

https://auth.globus.org/v2/oauth2/authorize?client_id=e6c75d97-532a-4c88-b031-8584a319fa3e&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=https%3A%2F%2Fauth.globus.org%2Fscopes%2F4b42bc90-951a-44ea-b761-93c3bf4f7b60%2Fflow_4b42bc90_951a_44ea_b761_93c3bf4f7b60_user&state=_default&response_type=code&code_challenge=4pc9pqiGUWw56yOFX1aTM34x1KPHqImdItW1i9xKlgE&code_challenge_method=S256&access_type=offline&prefill_named_grant=Globus+Automate+Client


Note that this link can only be used once! If login or a later step in the flow fails, you must restart it.
Enter resulting code:h82kcHNgp2d8wboaKHTNYKTuK6FohW
GlobusHTTPResponse({'action_id': 'b59660fa-7e75-40d5-b0dc-5a43ad4e407e', 'completion_time': 'None', 'created_by': 'urn:globus:auth:identity:c4765424-d274-11e5-b894-cb4139f74ecf', 'details': {'code': 'ActionStarted', 'description': 'State Transfer of type Action started', 'details': {'input': {'destination_endpoint_id': 'ddb59af0-6d04-11e5-ba46-22000b

Check the result of the analysis function

In [10]:
flow_action['details']['output']['AnalyzeResult']

{'action_id': '3a0c8539-e626-479a-9f43-5f8e6c52c44b',
 'details': {'completion_t': '1606755210.3358638',
  'exception': None,
  'result': 26,
  'status': 'SUCCEEDED',
  'task_id': '3a0c8539-e626-479a-9f43-5f8e6c52c44b'},
 'release_after': 'P30D',
 'state_name': 'Analyze',
 'status': 'SUCCEEDED'}